In [1]:
import nobuco
from nobuco import ChannelOrder, ChannelOrderingStrategy
from nobuco.layers.weight import WeightLayer

2024-07-23 03:26:31.791336: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-23 03:26:31.828038: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-23 03:26:31.828076: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-23 03:26:31.829229: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-23 03:26:31.835689: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructio

In [2]:
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet152
from torch.utils.data import DataLoader, Dataset
import numpy as np
import os

class ResNetBackbone(nn.Module):
    def __init__(self):
        super(ResNetBackbone, self).__init__()
        # Load a pre-trained ResNet-152 and remove the last GAP and FC
        base_model = resnet152(pretrained=True)
        self.features = nn.Sequential(*list(base_model.children())[:-2])

        # Additional blocks to reduce channel dimensions
        self.reduce_channels = nn.Sequential(
            nn.Conv2d(2048, 1024, kernel_size=3),
            nn.BatchNorm2d(1024),
            nn.ReLU(),
            nn.Conv2d(1024, 512, kernel_size=3),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.features(x)
        x = self.reduce_channels(x)
        return x


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ChannelAttention(nn.Module):
    def __init__(self, in_channels):
        super(ChannelAttention, self).__init__()
        self.in_channels = in_channels

    def forward(self, x):
        # x: input feature map with shape (batch_size, C, H, W)
        batch_size, C, H, W = x.size()
        
        # Reshape x to (batch_size, C, H*W)
        x_reshaped = x.view(batch_size, C, -1)
        
        # Compute the channel attention map
        channel_attention_map = torch.bmm(x_reshaped, x_reshaped.transpose(1, 2))
        channel_attention_map = F.softmax(channel_attention_map, dim=1)
        
        # Multiply the attention map by the input feature map
        x_weighted = torch.bmm(channel_attention_map, x_reshaped)
        
        # Reshape back to (batch_size, C, H, W)
        x_weighted = x_weighted.view(batch_size, C, H, W)
        
        # Apply scale parameter and element-wise summation
        # beta = torch.nn.Parameter(torch.zeros(1), requires_grad=False)
        beta = torch.zeros(1)
        out = beta * x_weighted + x
        
        return out

class SpatialAttention(nn.Module):
    def __init__(self, in_channels):
        super(SpatialAttention, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.conv2 = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.conv3 = nn.Conv2d(in_channels, in_channels, kernel_size=1)

    def forward(self, x):
        # x: input feature map with shape (batch_size, C, H, W)
        batch_size, Ch, H, W = x.size()
        
        # Obtain new feature maps B and C
        B = self.conv1(x)
        C = self.conv2(x)
        
        # Reshape B and C to (batch_size, C, H*W)
        B_reshaped = B.view(batch_size, Ch, -1)
        C_reshaped = C.view(batch_size, Ch, -1)
        
        # Compute the spatial attention map
        spatial_attention_map = torch.bmm(B_reshaped.transpose(1, 2), C_reshaped)
        spatial_attention_map = F.softmax(spatial_attention_map, dim=1)
        
        # Multiply the attention map by the input feature map
        D = self.conv3(x)
        D_reshaped = D.view(batch_size, Ch, -1)
        x_weighted = torch.bmm(spatial_attention_map, D_reshaped.transpose(1, 2))
        
        # Reshape back to (batch_size, C, H, W)
        x_weighted = x_weighted.view(batch_size, Ch, H, W)
        
        # Apply scale parameter and element-wise summation
        # alpha = torch.nn.Parameter(torch.zeros(1), requires_grad=False)
        alpha = torch.zeros(1)
        out = alpha * x_weighted + x
        
        return out

class DualAttentionNetwork(nn.Module):
    def __init__(self, in_channels):
        super(DualAttentionNetwork, self).__init__()
        self.channel_attention = ChannelAttention(in_channels)
        self.spatial_attention = SpatialAttention(in_channels)
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(in_channels * 3, 1024)

    def forward(self, x):
        # Apply channel attention
        channel_attention_map = self.channel_attention(x)
        
        # Apply spatial attention
        spatial_attention_map = self.spatial_attention(x)
        
        # Concatenate the input feature map with channel and spatial attention maps
        concatenated = torch.cat([x, channel_attention_map, spatial_attention_map], dim=1)
        
        # Global average pooling
        gap = self.global_avg_pool(concatenated)
        gap = gap.view(gap.size(0), -1)
        
        # Fully connected layer
        out = self.fc(gap)
        
        return out


In [4]:
class SiameseNetworkX(nn.Module):
    def __init__(self):
        super(SiameseNetworkX, self).__init__()
        self.backbone = ResNetBackbone()
        self.attention = DualAttentionNetwork(512)
        # self.pooling = nn.AdaptiveAvgPool2d(1)
        # self.fc = nn.Linear(512, 1024)  # Output 1024-dimensional embedding

    def forward(self, x1):
        out1 = self.backbone(x1)
        out1 = self.attention(out1)
        # out1 = self.pooling(out1)
        # out1 = out1.view(out1.size(0), -1)
        # out1 = self.fc(out1)


        # out2 = self.fc(out2)

        return out1


In [5]:

model = SiameseNetworkX()
model.load_state_dict(torch.load('/home/athena/Documents/GitHub/Dog-NosePrint-Recognition/best_acc_400.pth'))

/home/athena/anaconda3/envs/nobuCon/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/athena/anaconda3/envs/nobuCon/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [6]:
from torchvision import transforms
# Set the model to evaluation mode
# Change the device to CPU
# device = torch.device('cpu')
transform_val = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize the image to 256x256
    transforms.ToTensor(),          # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])



In [7]:
dummy_input = torch.randn(size=(1, 3, 256, 256))

In [8]:
for param in model.parameters():
    param.requires_grad = False

In [8]:
pytorch_module = model.eval()
with torch.no_grad():
    # output = pytorch_module(dummy_input)
    keras_model = nobuco.pytorch_to_keras(
        pytorch_module,
        args=[dummy_input], kwargs=None,
        inputs_channel_order=ChannelOrder.TENSORFLOW,
        outputs_channel_order=ChannelOrder.TENSORFLOW
)

[Nobuco] Tracing (DONE): 911 ops [00:01]
[Nobuco] Converting: |          | 3/544 ops [00:00]

2024-07-23 03:26:51.755506: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-23 03:26:51.758351: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


[Nobuco] Converting (DONE): |████████▏ | 444/544 ops [00:07]
Legend:
    Green — conversion successful
    Yellow — conversion imprecise
    Red — conversion failed
    Red — no converter found
    Bold — conversion applied directly
    * — subgraph reused
    Tensor — this output is not dependent on any of subgraph's input tensors
    Tensor — this input is a parameter / constant
    Tensor — this tensor is useless

SiameseNetworkX[__main__](float32_0<1,3,256,256>) -> float32_1140<1,1024>
 │  ResNetBackbone[__main__](float32_0<1,3,256,256>) -> float32_1104<1,512,4,4>
 │   │  Sequential[torch.nn.modules.container](float32_0<1,3,256,256>) -> float32_1086<1,2048,8,8>
 │   │   │  Conv2d[torch.nn.modules.conv](float32_0<1,3,256,256>) -> float32_2<1,64,128,128>
 │   │   │   └· conv2d[torch.nn.functional](float32_0<1,3,256,256>, float32_1<64,3,7,7>, None, (2, 2), (3, 3), (1, 1), 1) -> float32_2<1,64,128,128>
 │   │   │  BatchNorm2d[torch.nn.modules.batchnorm](float32_2<1,64,128,128>) -> floa

In [14]:
from PIL import Image
img_path = "/home/athena/Documents/GitHub/Dog-NosePrint-Recognition/dataset2/classes_with_6imgs/5672/O1CN01fjcDg01yHK0TN1bE3_!!0-mtopupload.jpg"
image = Image.open(img_path).convert('RGB')
transform_image = transform_val(image).unsqueeze(0)
print(transform_image.shape)
# convert it to 1,256,256,3
transform_image = transform_image.permute(0, 2, 3, 1)
keras_output = keras_model.predict(transform_image.numpy())

torch.Size([1, 3, 256, 256])
1/1 [==============================] - 0s 131ms/step


In [15]:
keras_output.shape

(1, 1024)

In [16]:
import numpy as np
np_vector = np.load('output.npy')

In [17]:
np_vector.shape

(1, 1024)

In [22]:
import numpy as np

def cosine_similarity(arr1, arr2):
    dot_product = np.dot(arr1, arr2.T)
    norm_a = np.linalg.norm(arr1)
    norm_b = np.linalg.norm(arr2)
    similarity = dot_product / (norm_a * norm_b)
    return similarity

cosine_similarity(keras_output, np_vector)


array([[1.]], dtype=float32)

In [24]:
keras_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(1, 256, 256, 3)]           0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (1, 262, 262, 3)             0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv2d (Conv2D)             (1, 128, 128, 64)            9408      ['zero_padding2d[0][0]']      
                                                                                                  
 batch_normalization (Batch  (1, 128, 128, 64)            256       ['conv2d[0][0]']          

In [27]:
keras_model.save('/home/athena/Documents/GitHub/Dog-NosePrint-Recognition/keras_model')

INFO:tensorflow:Assets written to: /home/athena/Documents/GitHub/Dog-NosePrint-Recognition/keras_model/assets


INFO:tensorflow:Assets written to: /home/athena/Documents/GitHub/Dog-NosePrint-Recognition/keras_model/assets


In [1]:
from tensorflow.keras.models import load_model

# Load the model
loaded_model = load_model('/home/athena/Documents/GitHub/Dog-NosePrint-Recognition/keras_model')

2024-07-23 15:38:57.939073: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-23 15:38:57.975108: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-23 15:38:57.975141: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-23 15:38:57.976308: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-23 15:38:57.982109: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-23 15:38:57.982748: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [31]:
loaded_model.predict(transform_image.numpy())

1/1 [==============================] - 1s 1s/step


array([[ -6.6147165, -13.800435 , -12.870659 , ...,  -1.7380959,
          9.257687 ,  -7.954967 ]], dtype=float32)

In [2]:
import tensorflow as tf

# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model
tflite_model_path = '/home/athena/Documents/GitHub/Dog-NosePrint-Recognition/keras_model_quant.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /tmp/tmpsvn891c4/assets


INFO:tensorflow:Assets written to: /tmp/tmpsvn891c4/assets
2024-07-23 15:39:47.780472: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-23 15:39:47.780510: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-23 15:39:47.781246: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpsvn891c4
2024-07-23 15:39:47.835980: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-23 15:39:47.836040: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpsvn891c4
2024-07-23 15:39:47.954233: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-07-23 15:39:48.013176: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-23 15:39:49.262473: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

In [39]:
from PIL import Image
# img_path = "/home/athena/Documents/GitHub/Dog-NosePrint-Recognition/dataset2/classes_with_6imgs/5672/O1CN01fjcDg01yHK0TN1bE3_!!0-mtopupload.jpg"
img_path2 = "dataset2/classes_with_6imgs/5981/A*3S3LTZgdkc9UCstAt954pwAAAQAAAQ.jpg"
image2 = Image.open(img_path2).convert('RGB')
transform_image2 = transform_val(image2).unsqueeze(0)
print(transform_image2.shape)
# convert it to 1,256,256,3
transform_image2 = transform_image2.permute(0, 2, 3, 1)
keras_output = keras_model.predict(transform_image2.numpy())

torch.Size([1, 3, 256, 256])
1/1 [==============================] - 0s 115ms/step


In [40]:
import numpy as np
import tensorflow as tf

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="/home/athena/Documents/GitHub/Dog-NosePrint-Recognition/keras_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare input data as a numpy array.
# Replace this with your actual input data. Ensure the shape matches the model's input shape.
input_data = transform_image2.numpy()

# Set the tensor to point to the input data to be inferred.
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run the inference.
interpreter.invoke()

# Get the output data.
output_data = interpreter.get_tensor(output_details[0]['index'])

# Print the output.
print("Inference Output:", output_data)


Inference Output: [[ -6.600711  -13.803787  -12.859123  ...  -1.7405246   9.242826
   -7.8949203]]


In [41]:
# find the cosine similarity between output_data and np_vector
cosine_similarity(output_data, np_vector)

array([[0.99999756]], dtype=float32)